# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle

# TODO: Fill this in based on where you saved the training and testing data

training_file = 'train.p'
validation_file= 'valid.p'
testing_file = 'test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results
import numpy as np
import pandas as pd

# TODO: Number of training examples
n_train = len(X_train)

# TODO: Number of validation examples
n_validation = len(X_valid)

# TODO: Number of testing examples.
n_test = len(X_test)

# TODO: What's the shape of an traffic sign image?
image_shape = X_train.shape[1:]

# TODO: How many unique classes/labels there are in the dataset.
n_classes = len(np.unique(np.concatenate([y_train, y_valid, y_test])))

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.

X_train_orig, X_valid_orig, X_test_orig = train['features'], valid['features'], test['features']

In [ ]:
sign_names_df = pd.read_csv('signnames.csv')
sign_label_names = sign_names_df.SignName.values

In [ ]:
# helper function to show images and their labels
import math

def show_images(images, labels, rows=1, fname=None):
    cols = math.ceil(len(images) / rows)
    fig, axis = plt.subplots(rows, cols)
    fig.set_size_inches(8, 8)
    for i in range(len(images)):
        row, col = i // cols, int(i % cols)
        if (rows > 1):
            axis[row,col].set_title(labels[i])
            axis[row,col].imshow(images[i])
        else:
            axis[i].set_title(labels[i])
            axis[i].imshow(images[i])
    if fname is not None:
        fig.savefig(fname)

In [ ]:
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline

np.random.seed(1)

idx = np.random.choice(n_train, 4)
print(idx)
signs = X_train_orig[idx]
sign_labels = y_train[idx]

show_images(signs, [sign_label_names[i] for i in sign_labels], rows=2, fname='train_sample.png')

In [ ]:
# Distribution of the total number of images per class

df = pd.DataFrame(train['labels'], columns=['label'])
data_count_per_class = df.groupby('label')['label'].count()
_plt = data_count_per_class.plot(kind='bar', legend=True, figsize=(20, 5), title="Count of training data per class")
_plt.figure.savefig('train_data_per_class.png')

### Augemnting training dataset

In this section I'm using keras data processing API to augment the training data in order to make the total number of items in each class equal. The random augmentation includes [-15..15] degree rotation, [-0.05..0.05] horizontal/vertical translation and [0.9..1.1] scaling

In [ ]:
max_count = data_count_per_class.max()

In [ ]:
from keras.preprocessing import image as process_img
from sklearn.utils import shuffle

datagen = process_img.ImageDataGenerator(
    featurewise_center=False,
    rotation_range=15, # [-15, 15] degree rotation
    width_shift_range=0.05, # [-0.05, 0.05] horizontal translation
    height_shift_range=0.05, # [-0.05, 0.05] vertical translation
    zoom_range=0.1,  # [0.9, 1.1] scaling
    data_format='channels_last')

datagen.fit(X_train)

In [ ]:
new_images, new_labels = [], []
for i in range(n_classes):
    image_indices = np.where(y_train == i)[0]
    expected = max_count - len(image_indices)
    for j in range(expected):
        intensity = np.random.uniform(0.6, 0.8)
        idx = j % len(image_indices)
        img = X_train[idx].astype(np.float32) / 255.
        new_img = datagen.random_transform(img)
        new_img = new_img * intensity
        new_images.append((new_img * 255.).astype(np.uint8))
        new_labels.append(i)

In [ ]:
new_image_arr, new_label_arr = shuffle(np.array(new_images), np.array(new_labels))

In [ ]:
new_image_arr.shape, new_label_arr.shape

In [ ]:
# do a sanity check of the total number of images
(X_train.shape[0] + new_image_arr.shape[0]) / max_count

In [ ]:
new_X_train = np.concatenate([new_image_arr, X_train])
new_y_train = np.concatenate([new_label_arr, y_train])

In [ ]:
new_X_train, new_y_train = shuffle(new_X_train, new_y_train)

In [ ]:
new_X_train.shape, new_y_train.shape

In [ ]:
# save the new train images
with open('new_train.p', mode='wb') as f:
    d = {'features': new_X_train, 'labels': new_y_train}
    pickle.dump(d, f)

In [ ]:
with open('new_train.p', mode='rb') as f:
    new_train = pickle.load(f)

In [ ]:
X_train, y_train = new_train['features'], new_train['labels']

In [ ]:
show_images(X_train[0:4], [sign_label_names[y_train[y]] for y in range(4)], rows=2)

In [ ]:
# keep a copy of original images for visualization
X_train_orig = new_train['features']

In [ ]:
# visualize the distribution of the number of images per class, now all images have exactly the same count
df = pd.DataFrame(new_train['labels'], columns=['label'])
data_count_per_class = df.groupby('label')['label'].count()
data_count_per_class.plot(kind='bar', legend=True, figsize=(20, 5))

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
X_train = 0.299 * X_train[:, :, :, 0] + 0.587 * X_train[:, :, :, 1] + 0.114 * X_train[:, :, :, 2]

In [ ]:
X_valid = 0.299 * X_valid[:, :, :, 0] + 0.587 * X_valid[:, :, :, 1] + 0.114 * X_valid[:, :, :, 2]

In [ ]:
X_test = 0.299 * X_test[:, :, :, 0] + 0.587 * X_test[:, :, :, 1] + 0.114 * X_test[:, :, :, 2]

In [ ]:
X_train = X_train[:,:,:,None]
X_valid = X_valid[:,:,:,None]
X_test = X_test[:,:,:,None]

In [ ]:
gray_scale = True;mu =0.0

In [ ]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.

# Convert to 0..1 float
X_train = X_train.astype(np.float32) * 1./255
X_valid = X_valid.astype(np.float32) * 1./255
X_test = X_test.astype(np.float32) * 1./255

# compute mean accross all images
# mu = np.mean(X_train, axis=(0,1,2))
# print(mu)

# # subtract training set mean from all of the sets
# X_train -= mu
# X_valid -= mu
# X_test -= mu

# if gray_scale:
#     X_train -= 0.5
#     X_valid -= 0.5
#     X_test -= 0.5

#     X_train = np.expand_dims(X_train[:,:,:,0], axis=-1)
#     X_valid = np.expand_dims(X_valid[:,:,:,0], axis=-1)
#     X_test = np.expand_dims(X_test[:,:,:,0], axis=-1)

In [ ]:
def normalize(X, mu):
    X = X.astype(np.float32) * 1./255
    X -= mu
    if gray_scale:
        X -= 0.5

        X = np.expand_dims(X[:,:,:,0], axis=-1)
    return X

In [ ]:
X_train.shape, X_valid.shape

In [ ]:
plt.imshow(X_train[0,:,:,0], cmap='gray')

### Model Architecture

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.
import tensorflow as tf

EPOCHS = 100
BATCH_SIZE = 128

In [ ]:
from tensorflow.contrib.layers import flatten

def LeNet2(x, keep_prob1 = None, keep_prob2 = None, keep_prob3 = None, keep_prob4 = None, grayscale = False):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    # keep a list of all activations
    activations = []
    
    num_channels = 1 if grayscale else 3
    
    # Layer 1: Convolutional. Input = 32x32x1. Output = 32x32x32.
    conv1_W = tf.Variable(tf.truncated_normal(mean=mu, stddev=sigma, shape=[5, 5, num_channels, 32]), name='conv1_W')
    conv1_b = tf.Variable(tf.zeros(32), name='conv1_b')
    x = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='SAME')
    x = tf.nn.bias_add(x, conv1_b)
    
    activations.append(x)

    # RELU Activation.
    x = tf.nn.relu(x)

    # Max Pooling. Input = 32x32x32. Output = 16x16x32.
    x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    x = tf.nn.dropout(x, keep_prob1)

    x1 = x

    # Layer 3: Convolutional. Output = 16x16x64.
    conv2_W = tf.Variable(tf.truncated_normal(mean=mu, stddev=sigma, shape=[5, 5, 32, 64]), name='conv2_W')
    conv2_b = tf.Variable(tf.zeros(64), name='conv2_b')
    x = tf.nn.conv2d(x, conv2_W, strides=[1, 1, 1, 1], padding='SAME')
    x = tf.nn.bias_add(x, conv2_b)
    
    activations.append(x)

    # Activation.
    x = tf.nn.relu(x)
    
    # Max Pooling. Input = 16x16x64. Output = 8x8x64.
    x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    x = tf.nn.dropout(x, keep_prob2)

    x2 = x

    # Layer 4: Convolutional. Output = 8x8x128.
    conv3_W = tf.Variable(tf.truncated_normal(mean=mu, stddev=sigma, shape=[5, 5, 64, 128]), name='conv3_W')
    conv3_b = tf.Variable(tf.zeros(128), name='conv3_b')
    x = tf.nn.conv2d(x, conv3_W, strides=[1, 1, 1, 1], padding='SAME')
    x = tf.nn.bias_add(x, conv3_b)
    
    activations.append(x)

    # Activation.
    x = tf.nn.relu(x)
    
    # Max Pooling. Input = 8x8x64. Output = 4x4x128.
    x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    x = tf.nn.dropout(x, keep_prob3)

    x1 = tf.nn.max_pool(x1, ksize=[1, 4, 4, 1], strides=[1, 4, 4, 1], padding='SAME')
    x2 = tf.nn.max_pool(x2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    # Flatten. Input = 4x4x128. Output = 5000.
    f1 = tf.contrib.layers.flatten(x1)
    f2 = tf.contrib.layers.flatten(x2)
    f3 = tf.contrib.layers.flatten(x)
    
    f = tf.concat([f1, f2, f3], 1)
        
    # Layer 3: Fully Connected. Input = 50000. Output = 1000.
    fc1_W = tf.Variable(tf.truncated_normal(mean=mu, stddev=sigma, shape=[f.get_shape().as_list()[-1], 1024]), name='fc1_W')
    fc1_b = tf.Variable(tf.zeros(1024), name='fc1_b')
    fc1 = tf.add(tf.matmul(f, fc1_W), fc1_b)

    # Activation.
    fc1 = tf.nn.relu(fc1)
    
    fc1 = tf.nn.dropout(fc1, keep_prob4)

    
    # Layer 5: Fully Connected. Input = 200. Output = n_classes.
    out_W = tf.Variable(tf.truncated_normal(mean=mu, stddev=sigma, shape=[1024, n_classes]), name='out_W')
    out_b = tf.Variable(tf.zeros(n_classes))    
    logits = tf.add(tf.matmul(fc1, out_W), out_b)
    
    return logits, activations

In [ ]:
# from tensorflow.contrib.layers import flatten

# def LeNet(x, keep_prob = None, grayscale = False):    
#     # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
#     mu = 0
#     sigma = 0.1
    
#     # keep a list of all activations
#     activations = []
    
#     num_channels = 1 if grayscale else 3
    
#     # Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x108.
#     conv1_W = tf.Variable(tf.truncated_normal(mean=mu, stddev=sigma, shape=[5, 5, num_channels, 108]), name='conv1_W')
#     conv1_b = tf.Variable(tf.zeros(108), name='conv1_b')
#     x = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID')
#     x = tf.nn.bias_add(x, conv1_b)
    
#     activations.append(x)

#     # RELU Activation.
#     x = tf.nn.relu(x)

#     # Max Pooling. Input = 28x28x108. Output = 14x14x108.
#     x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
#     # Layer 2: Convolutional. Output = 10x10x200.
#     conv2_W = tf.Variable(tf.truncated_normal(mean=mu, stddev=sigma, shape=[5, 5, 108, 200]), name='conv2_W')
#     conv2_b = tf.Variable(tf.zeros(200), name='conv2_b')
#     x = tf.nn.conv2d(x, conv2_W, strides=[1, 1, 1, 1], padding='VALID')
#     x = tf.nn.bias_add(x, conv2_b)
    
#     activations.append(x)

#     # Activation.
#     x = tf.nn.relu(x)
    
#     # Max Pooling. Input = 10x10x16. Output = 5x5x200.
#     x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
#     # Dropout
#     if keep_prob is not None:
#         x = tf.nn.dropout(x, keep_prob)

#     # Flatten. Input = 5x5x200. Output = 5000.
#     f = tf.contrib.layers.flatten(x)
        
#     # Layer 3: Fully Connected. Input = 50000. Output = 1000.
#     fc1_W = tf.Variable(tf.truncated_normal(mean=mu, stddev=sigma, shape=[5*5*200, 1000]), name='fc1_W')
#     fc1_b = tf.Variable(tf.zeros(1000), name='fc1_b')
#     fc1 = tf.add(tf.matmul(f, fc1_W), fc1_b)

#     # Activation.
#     fc1 = tf.nn.relu(fc1)

#     # Layer 4: Fully Connected. Input = 1000. Output = 200.
#     fc2_W = tf.Variable(tf.truncated_normal(mean=mu, stddev=sigma, shape=[1000, 200]), name='fc2_W')
#     fc2_b = tf.Variable(tf.zeros(200), name='fc2_b')
#     fc2 = tf.add(tf.matmul(fc1, fc2_W), fc2_b)
    
#     # Activation.
#     fc2 = tf.nn.relu(fc2)
    
#     # Layer 5: Fully Connected. Input = 200. Output = n_classes.
#     out_W = tf.Variable(tf.truncated_normal(mean=mu, stddev=sigma, shape=[200, n_classes]), name='out_W')
#     out_b = tf.Variable(tf.zeros(n_classes))    
#     logits = tf.add(tf.matmul(fc2, out_W), out_b)
    
#     return logits, activations

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
rate_value = 0.001
drop_out_prob = 0.5
use_dropout = True

# use learning rate decay to lower the learning rate as learning progresses
adaptive_lr = True

In [ ]:
tf.reset_default_graph()

num_channels = 1 if gray_scale else 3

x = tf.placeholder(dtype=tf.float32, shape=(None, 32, 32, num_channels))
y = tf.placeholder(dtype=tf.int32, shape=(None))

if adaptive_lr:
    global_step = tf.Variable(0, trainable=False)
    starter_learning_rate = 0.001
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                               5000, 0.96, staircase=False)

one_hot_y = tf.one_hot(y, n_classes)
if use_dropout:
    keep_prob = tf.placeholder(tf.float32)
    
    keep_prob1 = tf.placeholder(tf.float32)
    keep_prob2 = tf.placeholder(tf.float32)
    keep_prob3 = tf.placeholder(tf.float32)
    keep_prob4 = tf.placeholder(tf.float32)

In [ ]:
logits, activations = LeNet2(x, keep_prob1=keep_prob1, keep_prob2=keep_prob2, keep_prob3=keep_prob3, keep_prob4=keep_prob4, grayscale=gray_scale)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=one_hot_y)
loss_op = tf.reduce_mean(cross_entropy)

if adaptive_lr:
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=global_step)
else:
    optimizer = tf.train.AdamOptimizer(learning_rate=rate_value)
    train_op = optimizer.minimize(loss_op)

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

In [ ]:
def evaluate(sess, X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        # to evaluate we shutdown dropout
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob1: 1, keep_prob2: 1, keep_prob3: 1, keep_prob4: 1})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [ ]:
def plot_history(history):
    epochs = len(history)
    plt.plot([history[i][0] for i in range(epochs)], label='train')
    plt.plot([history[i][1] for i in range(epochs)], label='valid')
    plt.title('Accuracy')
    plt.legend(['train', 'test'])
    plt.savefig('history.png')

In [ ]:
import os
from sklearn.utils import shuffle
from keras.preprocessing import image as process_img

if not os.path.exists('models2'): os.mkdir('models2')
    
history = []

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
        
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, n_train, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            feed_dict = {x: batch_x, y: batch_y, keep_prob1: 0.9, keep_prob2: 0.8, keep_prob3: 0.7, keep_prob4: 0.5}
            sess.run(train_op, feed_dict=feed_dict)
            
        print("Learning rate {0:.6f}".format(learning_rate.eval()))
        train_accuracy = evaluate(sess, X_train, y_train)
        validation_accuracy = evaluate(sess, X_valid, y_valid)
        
        history.append((train_accuracy, validation_accuracy))
        print("EPOCH {} ...".format(i+1))
        print("Train Accuracy = {:.3f}, Validation Accuracy = {:.3f}".format(train_accuracy, validation_accuracy))
        print()
    
    saver.save(sess, 'models2/lenet')

In [ ]:
plot_history(history)

### Evalue Model on validation set and visualization

In [ ]:
def predict(image):
    with tf.Session() as sess:
        saver.restore(sess, tf.train.latest_checkpoint('models'))

        x_data = np.array(image).astype(np.float32)
        if len(x_data.shape) == 3:
            x_data = x_data[None,:]

        pred = sess.run(logits, feed_dict={x: x_data, keep_prob: 1.0})
        return np.argmax(pred, axis=1)

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('models'))

    valid_accuracy = evaluate(sess, X_valid, y_valid)
    print("Validation Accuracy = {:.3f}".format(valid_accuracy))
    
    np.random.seed(0)
    
    idx = np.random.choice(n_validation, 4)
    x_data = X_valid[idx]
    y_data = y_valid[idx]
    images = X_valid_orig[idx]
    pred = sess.run(logits, feed_dict={x: x_data, keep_prob: 1.0})
    pred_class = sign_label_names[np.argmax(pred, axis=1)]
    actual_class = sign_label_names[y_data]
    
    show_images(images, [p+'/'+a for p,a in zip(pred_class, actual_class)], rows=2, fname='sample_validation_prediction.png')

### Evaluate model on test data and visualization

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('models'))

    test_accuracy = evaluate(sess, X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))
    
    np.random.seed(0)
    
    idx = np.random.choice(n_test, 4)
    x_data = X_test[idx]
    y_data = y_test[idx]
    images = X_test_orig[idx]
    pred = sess.run(logits, feed_dict={x: x_data, keep_prob: 1.0})
    pred_class = sign_label_names[np.argmax(pred, axis=1)]
    actual_class = sign_label_names[y_data]
    
    show_images(images, [p+'/'+a for p,a in zip(pred_class, actual_class)], rows=2, fname='sample_test_prediction.png')

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.
import matplotlib.image as mpimg
import cv2

image_files = [
    'double_curve.jpg',
    'speed_30.jpg',
    'pedestrians-2.jpg',
    'bump.png',
    'pedestrians.jpg'
]

images = []
for f in image_files:
    image = cv2.imread(f)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image,(32, 32), interpolation = cv2.INTER_CUBIC)
    # remove last channel if any
    if image.shape[2] > 3:
        image = image[:,:,0:3]
    images.append(image)
    
images = np.array(images)
images_orig = images.copy()

show_images(images, image_files, rows=2)

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

images = images.astype(np.float32) / 255.
images -= mu
# normalize(images, mu)

pred_classes = predict(images)

expected_classes = np.array([41,9,17,14,12])

# the prediction is shown as the label of the image
show_images(images_orig, [sign_label_names[pred_class] for i, pred_class in enumerate(pred_classes)], rows=2)

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('models'))

    accuracy = evaluate(sess, images, expected_classes)
    print("Test image Accuracy = {:.3f}".format(accuracy))

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.
top_5 = tf.nn.top_k(logits, k=5)

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('models'))

    x_data = images
    top_5_value = sess.run(top_5, feed_dict={x: x_data, keep_prob: 1.0})

In [ ]:
top_5_value

In [ ]:
fig, axis = plt.subplots(5, 1)
fig.set_size_inches(10, 10)
indices = top_5_value.indices
for i in range(5):
    axis[i].imshow(images[i])
    for j in range(5):
        bbox = {'alpha':0.5, 'pad':5}
        if indices[i,j] == expected_classes[i]:
            color = 'red' if j > 0 else 'green'
            bbox['facecolor'] = color
        elif j == 0:
            bbox['facecolor'] = 'red'
        axis[i].text(35, j*7, sign_label_names[indices[i][j]], bbox=bbox)

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry
import math

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        rows = int(math.ceil(featuremaps / 8))
        plt.subplot(rows,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")

In [ ]:
np.random.seed(100)
random_image_idx = np.random.choice(len(X_test_orig))
image_input = [X_test[random_image_idx]]

plt.imshow(X_test_orig[random_image_idx])

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('models'))
    
    outputFeatureMap(image_input, activations[0])

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('models'))
    
    outputFeatureMap(image_input, activations[1])